# ラッソ回帰（Lasso Regression）
回帰モデルの重みをスパース化し、特徴選択を自動で行う線形回帰手法
### スパース化
データの要素の多くをゼロにして、必要な情報だけを残すこと  
機械学習では、モデルのパラメータ（重み）をスパース化することで、不要な特徴量の影響をゼロにする 
### メリット
特徴選択ができる（不要な特徴の重みが 0 になる）  
過学習を抑制できる  
計算コストが低い
### デメリット
高次元データでは情報を失う可能性がある  
変数の数が多すぎると計算がやや重くなる  
非線形関係を学習できない


In [9]:
import torch
import torch.nn as nn

#ラッソ回帰モデル（単純な線形回帰）
class LassoRegression(nn.Module):
    def __init__(self, input_dim):
        super(LassoRegression, self).__init__()
        self.linear = nn.Linear(input_dim, 1)

    def forward(self, x):
        return self.linear(x)

In [11]:

import torch.optim as optim
from sklearn.datasets import make_regression
from sklearn.preprocessing import StandardScaler

# **データの準備**
X, y = make_regression(n_samples=100, n_features=10, noise=0.1, random_state=42)
scaler = StandardScaler()
X = scaler.fit_transform(X)
y = (y - y.mean()) / y.std()  # 正規化

X = torch.tensor(X, dtype=torch.float32)
y = torch.tensor(y, dtype=torch.float32).view(-1, 1)



model = LassoRegression(input_dim=X.shape[1])

# **損失関数 & 最適化（L1 正則化適用）**
criterion = nn.MSELoss()
optimizer = optim.SGD(model.parameters(), lr=0.01)

lambda_l1 = 0.01  # L1 正則化係数

# **学習**
num_epochs = 100
for epoch in range(num_epochs):
    optimizer.zero_grad()
    outputs = model(X)
    loss = criterion(outputs, y)
    
    # L1 正則化を手動で適用
    l1_norm = sum(p.abs().sum() for p in model.parameters())
    loss += lambda_l1 * l1_norm

    loss.backward()
    optimizer.step()

    if (epoch + 1) % 10 == 0:
        print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}")


Epoch [10/100], Loss: 0.9925
Epoch [20/100], Loss: 0.6456
Epoch [30/100], Loss: 0.4328
Epoch [40/100], Loss: 0.2979
Epoch [50/100], Loss: 0.2107
Epoch [60/100], Loss: 0.1535
Epoch [70/100], Loss: 0.1153
Epoch [80/100], Loss: 0.0893
Epoch [90/100], Loss: 0.0714
Epoch [100/100], Loss: 0.0588
